# Grouping Data
*This modulewe will learn how we can find out many different answers to quesions using just one dataset span across multiple parameters.
## Contents
  * Downloading data
  * How to group on different parameters for multiple results.
  * Getting the Comtrade Data
  * Pattern of Split-Apply-Combine
  * Filtering the data


In [0]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')

## Downloading Data
* We are going to download using pandas <pre>download()</pre> 
* The GDP of UK, China from the year 2008 to 2013.

In [0]:
from pandas_datareader import wb
YEAR = 2013 #Constant variables.
GDP_INDICATOR = 'NY.GDP.MKTP.CD'
data = wb.download(indicator = GDP_INDICATOR, country=['GB','CN'], start=YEAR-5, end=YEAR) 
data = data.reset_index() #reseting index will reset the indexex into columns.
data

,country,year,NY.GDP.MKTP.CD
0,China,2013,9.570406e+12
1,China,2012,8.532231e+12
2,China,2011,7.551500e+12
3,China,2010,6.087165e+12
4,China,2009,5.101702e+12
5,China,2008,4.594307e+12
6,United Kingdom,2013,2.786023e+12
7,United Kingdom,2012,2.704888e+12
8,United Kingdom,2011,2.659310e+12
9,United Kingdom,2010,2.475244e+12


## Grouping Data
* Grouping allows us to analyze data onto many frontiers.
* We will use pandas's <pre>Groupby(<i>column_name</i>)</pre> for grouping data columns.
* To apply aggregate functions on the data we will use <pre>pandas.Dataframe.<b>aggregate()</b></pre>

In [0]:
#Find the total sum of GDP for each country over all the years.
data.groupby('country')['NY.GDP.MKTP.CD'].aggregate(sum)

country
China             4.143731e+13
United Kingdom    1.595904e+13
Name: NY.GDP.MKTP.CD, dtype: float64

In [0]:
#Total combined GDP of two countries in each year.
data.groupby('year')[GDP_INDICATOR].aggregate(sum)

year
2008    7.516974e+12
2009    7.512612e+12
2010    8.562409e+12
2011    1.021081e+13
2012    1.123712e+13
2013    1.235643e+13
Name: NY.GDP.MKTP.CD, dtype: float64

## Getting the Comtrade data
* Comtrade database is UN's import export data base of the world.
* You will learn to fetch data from the URL itself usning Comtrade API.

In [0]:
URL='http://comtrade.un.org/api/get?max=5000&type=C&freq=A&px=HS&ps=2014%2C2013%2C2012&r=826&p=all&rg=all&cc=0401%2C0402&fmt=csv'

df=pd.read_csv(URL, dtype={'Commodity Code':str, 'Reporter Code':str}) #here Commodity code is given in str becaue it will return 401 for 0401 otherwise.

In [0]:
#To save a copy of this data file to your folder we use to_csv() function.
df.to_csv('Data_copy.csv', index=False) # Index=false is to aviod pandas to assign its default index in the new file. 
#check the Files(Folder symbol) tab on the left hand side and you will find the copied file.

In [0]:
df.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Customs Proc. Code,Customs,Mode of Transport Code,Mode of Transport,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Alt Qty Unit Code,Alt Qty Unit,Alt Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
0,H4,2012,2012,2012,4,0,1,Import,826,United Kingdom,GBR,0,World,WLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0401,"Milk and cream; not concentrated, not containi...",8,Weight in kilograms,187245794,NaN,NaN,NaN,187245794,NaN,177458632,NaN,NaN,0
1,H4,2012,2012,2012,4,0,2,Export,826,United Kingdom,GBR,0,World,WLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0401,"Milk and cream; not concentrated, not containi...",8,Weight in kilograms,614271513,NaN,NaN,NaN,614271513,NaN,399077664,NaN,NaN,6
2,H4,2012,2012,2012,4,0,2,Export,826,United Kingdom,GBR,4,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0401,"Milk and cream; not concentrated, not containi...",8,Weight in kilograms,33841,NaN,NaN,NaN,33841,NaN,70986,NaN,NaN,0
3,H4,2012,2012,2012,4,0,2,Export,826,United Kingdom,GBR,24,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0401,"Milk and cream; not concentrated, not containi...",8,Weight in kilograms,22247,NaN,NaN,NaN,22247,NaN,19448,NaN,NaN,0
4,H4,2012,2012,2012,4,0,2,Export,826,United Kingdom,GBR,36,Australia,AUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0401,"Milk and cream; not concentrated, not containi...",8,Weight in kilograms,960,NaN,NaN,NaN,960,NaN,13515,NaN,NaN,0


## Pattern of Split-apply-combine
* To perform operations on hetereogenous dataset and to figure an answer to a particular question  then this pattern is used.
* We create a smaller dataset for understanding.

In [0]:
#Creating a smaller data
data=[['A',10],['A',15],['A',5],['A',20],

              ['B',10],['B',10],['B',5],

              ['C',20],['C',30]] 

df=pd.DataFrame(data=data, columns=["Commodity","Amount"])
df

,Commodity,Amount
0,A,10
1,A,15
2,A,5
3,A,20
4,B,10
5,B,10
6,B,5
7,C,20
8,C,30


In [0]:
#Splitting the data or grouping.
grouped_data = df.groupby('Commodity')
grouped_data

In [0]:
#performing operations
grouped_data.groups.keys()

dict_keys(['A', 'B', 'C'])

In [0]:
grouped_data.get_group('A') #returns only A commodity data.

,Commodity,Amount
0,A,10
1,A,15
2,A,5
3,A,20


In [0]:
#Applying the sumary or aggregation on the group.
grouped_data.aggregate(sum) #this calculates the sum of each groupand combines the result in a flat table.

,Amount
Commodity,
A,50
B,25
C,50


In [0]:
def top2ByAmount(g):
  return g.sort_values('Amount', ascending=False).head(2) #Display only top 2 amounts.
grouped_data.apply(top2ByAmount)

Amount
Commodity          
A         3      20
          1      15
B         4      10
          5      10
C         8      30
          7      20

## Filtering the data 
* Sometimes for analysis there is only a need of the subset of the data fromteh dataset.
* There may be requirement of only that groups that satisfy some constraints.
* For filtering pandas has <pre>filter()</pre>
* The filter() method uses a function that returns a boolean ( True or False ) value to decide whether or not to filter through the rows associated with a particular group.

In [0]:
#Getting the groups with atmost 3 rows.
def groupsOfAtMost3Rows(g):
  return len(g) <= 3

In [0]:
grouped_data.aggregate(len)

,Amount
Commodity,
A,4
B,3
C,2


In [0]:
grouped_data.filter(groupsOfAtMost3Rows)

,Commodity,Amount
4,B,10
5,B,10
6,B,5
7,C,20
8,C,30
